In [1]:
import pandas as pd
import jpy_tools.parseSnake2 as jps

In [2]:
configPath = '/public/home/liuzj/scripts/pipeline/10xAnalysisBySTARsolo/snakemake/config.yaml'
snakePath = '/public/home/liuzj/scripts/pipeline/10xAnalysisBySTARsolo/snakemake/snakefile'

In [3]:
snakeFile = jps.SnakeFile()

In [4]:
snakeHeader = jps.SnakeHeader(snakeFile, configPath)
snakeHeader.addCode("sampleLs = list(config['samples'].keys())")
config = snakeHeader.getConfig()
snakeHeader

/public/home/liuzj/softwares/anaconda3/lib/python3.8/site-packages/jpy_tools/parseSnake2.py:93: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.yaml = yaml.load(open(self.path))


import pandas as pd
#configfile: "/public/home/liuzj/scripts/pipeline/10xAnalysisBySTARsolo/snakemake/config.yaml"
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"
sampleLs = list(config['samples'].keys())

In [5]:
import glob
df_runStarsolo = pd.DataFrame(config['samples']).T
df_runStarsolo = df_runStarsolo.assign(starIndex = config['starIndex'], outputDir=df_runStarsolo.index + '/')
df_runStarsolo['sampleR2'] = df_runStarsolo['sampleDir'].map(lambda x:','.join(glob.glob(f"{x}/*/*R2*.fastq.gz")))
df_runStarsolo['sampleR1'] = df_runStarsolo['sampleDir'].map(lambda x:','.join(glob.glob(f"{x}/*/*R1*.fastq.gz")))

In [6]:
runStarsolo = jps.SnakeRule(snakeFile, "runStarsolo", 32, priority=20)
runStarsolo.addCode(
    """
import glob
df_runStarsolo = pd.DataFrame(config['samples']).T
df_runStarsolo = df_runStarsolo.assign(starIndex = config['starIndex'], outputDir=df_runStarsolo.index + '/')
df_runStarsolo['sampleR2'] = df_runStarsolo['sampleDir'].map(lambda x:','.join(glob.glob(f"{x}/*/*R2*.fastq.gz")))
df_runStarsolo['sampleR1'] = df_runStarsolo['sampleDir'].map(lambda x:','.join(glob.glob(f"{x}/*/*R1*.fastq.gz")))
"""
)
runStarsolo.addMetaDf(
    "df_runStarsolo",
    ["outputDir"]
)
runStarsolo.addMain('input', ['sampleDir'])
runStarsolo.addMain('params', ['expectedCell', 'umiLength', 'whiteList', 'includeIntron', 'starIndex', 'outputDir', 'sampleR2', 'sampleR1'])
runStarsolo.setShell("""
mkdir -p {params.outputDir}
cd {params.outputDir}
if [ {params.includeIntron} = True ]
then
    STAR --runThreadN {threads} --genomeDir {params.starIndex} --soloType CB_UMI_Simple --soloCBwhitelist {params.whiteList}\
        --soloUMIlen {params.umiLength} --readFilesIn {params.sampleR2} {params.sampleR1} --soloFeatures Gene GeneFull Velocyto\
        --clipAdapterType CellRanger4 --outFilterScoreMin 30 --soloCBmatchWLtype 1MM_multi_Nbase_pseudocounts --soloUMIfiltering MultiGeneUMI_CR\
        --soloUMIdedup 1MM_CR --soloCellFilter  EmptyDrops_CR {params.expectedCell} 0.99 10 45000 90000 500 0.01 20000 0.01 10000 \
        --readFilesCommand zcat
else
    STAR --runThreadN {threads} --genomeDir {params.starIndex} --soloType CB_UMI_Simple --soloCBwhitelist {params.whiteList}\
        --soloUMIlen {params.umiLength} --readFilesIn {params.sampleR2} {params.sampleR1} --soloFeatures Gene Velocyto\
        --clipAdapterType CellRanger4 --outFilterScoreMin 30 --soloCBmatchWLtype 1MM_multi_Nbase_pseudocounts --soloUMIfiltering MultiGeneUMI_CR\
        --soloUMIdedup 1MM_CR --soloCellFilter  EmptyDrops_CR {params.expectedCell} 0.99 10 45000 90000 500 0.01 20000 0.01 10000 \
        --readFilesCommand zcat
fi
samtools view -@ {threads} -S -O BAM ./Aligned.out.sam  | samtools sort -@ {threads} - -O BAM -o ./Aligned.out.bam
samtools index -@ {threads} ./Aligned.out.bam
rm ./Aligned.out.sam
""")
runStarsolo

2021-07-12 18:52:58.074 | INFO     | jpy_tools.parseSnake2:addRule:55 - runStarsolo step num: 1


import glob
df_runStarsolo = pd.DataFrame(config['samples']).T
df_runStarsolo = df_runStarsolo.assign(starIndex = config['starIndex'], outputDir=df_runStarsolo.index + '/')
df_runStarsolo['sampleR2'] = df_runStarsolo['sampleDir'].map(lambda x:','.join(glob.glob(f"{x}/*/*R2*.fastq.gz")))
df_runStarsolo['sampleR1'] = df_runStarsolo['sampleDir'].map(lambda x:','.join(glob.glob(f"{x}/*/*R1*.fastq.gz")))
for column in ['outputDir']:
    df_runStarsolo[column] = resultDir + 'step1_runStarsolo/' + df_runStarsolo[column]
----------------
IN RULE
----------------
rule runStarsolo:
    input:
        sampleDir = lambda wildcard: df_runStarsolo.at[wildcard.sample, 'sampleDir'],
    output:
        runStarsoloFinished = resultDir + 'step1_runStarsolo/' + '{sample}.finished',
    params:
        gpu = 0,
        expectedCell = lambda wildcard: df_runStarsolo.at[wildcard.sample, 'expectedCell'],
        umiLength = lambda wildcard: df_runStarsolo.at[wildcard.sample, 'umiLength'],
        whiteList =

In [7]:
import glob
df_runStarsoloWithAt = pd.DataFrame(config['samples']).T
df_runStarsoloWithAt = df_runStarsoloWithAt.assign(starIndex = config['starIndex'], outputDir=df_runStarsoloWithAt.index + '/')
df_runStarsoloWithAt['sampleR2'] = df_runStarsoloWithAt['sampleDir'].map(lambda x:','.join(glob.glob(f"{x}/*/*R2*.fastq.gz")))
df_runStarsoloWithAt['sampleR1'] = df_runStarsoloWithAt['sampleDir'].map(lambda x:','.join(glob.glob(f"{x}/*/*R1*.fastq.gz")))

In [8]:
runStarsoloWithAt = jps.SnakeRule(snakeFile, "runStarsoloWithAt", 32)
runStarsoloWithAt.addCode(
    """
import glob
df_runStarsoloWithAt = pd.DataFrame(config['samples']).T
df_runStarsoloWithAt = df_runStarsoloWithAt.assign(starIndex = config['starIndex'], outputDir=df_runStarsoloWithAt.index + '/')
df_runStarsoloWithAt['sampleR2'] = df_runStarsoloWithAt['sampleDir'].map(lambda x:','.join(glob.glob(f"{x}/*/*R2*.fastq.gz")))
df_runStarsoloWithAt['sampleR1'] = df_runStarsoloWithAt['sampleDir'].map(lambda x:','.join(glob.glob(f"{x}/*/*R1*.fastq.gz")))
"""
)
runStarsoloWithAt.addMetaDf(
    "df_runStarsoloWithAt",
    ["outputDir"]
)
runStarsoloWithAt.addMain('input', ['sampleDir'])
runStarsoloWithAt.addMain('params', ['expectedCell', 'umiLength', 'whiteList', 'includeIntron', 'starIndex', 'outputDir', 'sampleR2', 'sampleR1'])
runStarsoloWithAt.setShell("""
mkdir -p {params.outputDir}
cd {params.outputDir}
if [ {params.includeIntron} = True ]
then
    STAR --runThreadN {threads} --genomeDir {params.starIndex} --soloType CB_UMI_Simple --soloCBwhitelist {params.whiteList}\
        --soloUMIlen {params.umiLength} --readFilesIn {params.sampleR2} {params.sampleR1} --soloFeatures Gene GeneFull Velocyto\
        --clipAdapterType CellRanger4 --outFilterScoreMin 30 --soloCBmatchWLtype 1MM_multi_Nbase_pseudocounts --soloUMIfiltering MultiGeneUMI_CR\
        --soloUMIdedup 1MM_CR --soloCellFilter  EmptyDrops_CR {params.expectedCell} 0.99 10 45000 90000 500 0.01 20000 0.01 10000 \
        --readFilesCommand zcat --soloStrand Unstranded
else
    STAR --runThreadN {threads} --genomeDir {params.starIndex} --soloType CB_UMI_Simple --soloCBwhitelist {params.whiteList}\
        --soloUMIlen {params.umiLength} --readFilesIn {params.sampleR2} {params.sampleR1} --soloFeatures Gene Velocyto\
        --clipAdapterType CellRanger4 --outFilterScoreMin 30 --soloCBmatchWLtype 1MM_multi_Nbase_pseudocounts --soloUMIfiltering MultiGeneUMI_CR\
        --soloUMIdedup 1MM_CR --soloCellFilter  EmptyDrops_CR {params.expectedCell} 0.99 10 45000 90000 500 0.01 20000 0.01 10000 \
        --readFilesCommand zcat --soloStrand Unstranded
fi
samtools view -@ {threads} -S -O BAM ./Aligned.out.sam  | samtools sort -@ {threads} - -O BAM -o ./Aligned.out.bam
samtools index -@ {threads} ./Aligned.out.bam
rm ./Aligned.out.sam
""")
runStarsoloWithAt

2021-07-12 18:53:07.521 | INFO     | jpy_tools.parseSnake2:addRule:55 - runStarsoloWithAt step num: 2


import glob
df_runStarsoloWithAt = pd.DataFrame(config['samples']).T
df_runStarsoloWithAt = df_runStarsoloWithAt.assign(starIndex = config['starIndex'], outputDir=df_runStarsoloWithAt.index + '/')
df_runStarsoloWithAt['sampleR2'] = df_runStarsoloWithAt['sampleDir'].map(lambda x:','.join(glob.glob(f"{x}/*/*R2*.fastq.gz")))
df_runStarsoloWithAt['sampleR1'] = df_runStarsoloWithAt['sampleDir'].map(lambda x:','.join(glob.glob(f"{x}/*/*R1*.fastq.gz")))
for column in ['outputDir']:
    df_runStarsoloWithAt[column] = resultDir + 'step2_runStarsoloWithAt/' + df_runStarsoloWithAt[column]
----------------
IN RULE
----------------
rule runStarsoloWithAt:
    input:
        sampleDir = lambda wildcard: df_runStarsoloWithAt.at[wildcard.sample, 'sampleDir'],
    output:
        runStarsoloWithAtFinished = resultDir + 'step2_runStarsoloWithAt/' + '{sample}.finished',
    params:
        gpu = 0,
        expectedCell = lambda wildcard: df_runStarsoloWithAt.at[wildcard.sample, 'expectedCell'],
        u

In [9]:
snakeAll = jps.SnakeAll(snakeFile, runStarsolo, runStarsoloWithAt)
snakeAll

rule all:
    input:
        runStarsoloFinished = [resultDir + 'step1_runStarsolo/' + "" + sample + ".finished" for sample in sampleLs],
        runStarsoloWithAtFinished = [resultDir + 'step2_runStarsoloWithAt/' + "" + sample + ".finished" for sample in sampleLs],

In [13]:
snakeFile.getMain(snakePath)

import pandas as pd
#configfile: "/public/home/liuzj/scripts/pipeline/10xAnalysisBySTARsolo/snakemake/config.yaml"
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"
sampleLs = list(config['samples'].keys())
import glob
df_runStarsolo = pd.DataFrame(config['samples']).T
df_runStarsolo = df_runStarsolo.assign(starIndex = config['starIndex'], outputDir=df_runStarsolo.index + '/')
df_runStarsolo['sampleR2'] = df_runStarsolo['sampleDir'].map(lambda x:','.join(glob.glob(f"{x}/*/*R2*.fastq.gz")))
df_runStarsolo['sampleR1'] = df_runStarsolo['sampleDir'].map(lambda x:','.join(glob.glob(f"{x}/*/*R1*.fastq.gz")))
for column in ['outputDir']:
    df_runStarsolo[column] = resultDir + 'step1_runStarsolo/' + df_runStarsolo[column]

import glob
df_runStarsoloWithAt = pd.DataFrame(config['samples']).T
df_runStarsoloWithAt = df_runStarsoloWithAt.assign(starIndex = config['starIndex'], outputDir=df_runStarsoloWithAt

In [12]:
snakePath

'/public/home/liuzj/scripts/pipeline/10xAnalysisBySTARsolo/snakemake/snakefile'